REFRENCES 

https://www.kaggle.com/piantic/siim-fisabio-rsna-covid-19-detection-basic-eda/notebook#7.-Etc.---Pandas-Profiling-%F0%9F%8C%A4%EF%B8%8F by @Heroseo

https://www.kaggle.com/andradaolteanu/siim-covid-19-3d-analysis-augmentations by @andradaolteanu

https://www.kaggle.com/piantic/siim-fisabio-rsna-covid-19-detection-basic-eda by @piantic

https://www.kaggle.com/tanlikesmath/siim-covid-19-detection-a-simple-eda by @tanlikesmath



In [ ]:
import glob,tqdm
import pandas as pd,numpy as np,os,ast,seaborn as sns,matplotlib.pyplot as plt,matplotlib as mpl,cv2
# !pip install plotly
import plotly_express as px
from colorama import Fore, Back, Style
import pydicom as dicom
sample_sub_loc = '../input/siim-covid19-detection/sample_submission.csv'
image_level_loc = '../input/siim-covid19-detection/train_image_level.csv'
study_level_loc  = '../input/siim-covid19-detection/train_study_level.csv'
train_images_loc = '../input/siim-covid19-detection/train'
test_images_loc = '../input/siim-covid19-detection/test'

In [ ]:
print('Number of Train Images',len(glob.glob(os.path.join(train_images_loc,'*','*','*.dcm'))),'Test Images',len(glob.glob(os.path.join(test_images_loc,'*','*','*'))))
print(len(glob.glob(os.path.join(train_images_loc,'*'))),len(glob.glob(os.path.join(train_images_loc,'*','*'))))

Study level


6054 IDs(Points to a unique dcm)

Single class multiple options


In [ ]:
study_level = pd.read_csv(study_level_loc)
study_level['id'] = study_level['id'].str.split('_',expand=True)[0]
print('Shape of study' ,study_level.shape,'Check for Nulls ',study_level.isnull().sum(),sep='\n')
print('Unique IDS are ',study_level['id'].nunique())

# glob.glob(os.path.join(train_images_loc,'00086460a852','*','*.dcm'))

viz = study_level[['Negative for Pneumonia', 'Typical Appearance','Indeterminate Appearance', 'Atypical Appearance']].idxmax(axis=1)
study_level['type'] = study_level[['Negative for Pneumonia', 'Typical Appearance','Indeterminate Appearance', 'Atypical Appearance']].idxmax(axis=1)
study_level['Images_in_study_file']=study_level['id'].apply(lambda x:len(glob.glob(os.path.join(train_images_loc,x,'*','*.dcm'))))
display(study_level.head())
plt.pie(viz.value_counts(),labels=viz.value_counts().index,autopct=lambda x:f"{x:.2f}% ")

In [ ]:
px.histogram(study_level['Images_in_study_file'],histnorm='percent')

In [ ]:
for i in study_level['id'].iteritems():
    if len(glob.glob(os.path.join(train_images_loc,'00086460a852','*','*.dcm')))>1:
        print(i[1]) 
# Hence study id 1:1 image

Image level 

Same length as the number of images we have.All ids are unique.

Some Images have no boxes or NAN with label none 1 0 0 1 1 .

To access a picture we have to use study instance uid which are on many:1 relation with id.

In [ ]:
image_level = pd.read_csv(image_level_loc)
image_level[['id','t']]= image_level['id'].str.split('_',expand=True)
print('Shape of Image level',image_level.shape,'Checking Unique IDS and also (Study Instance ID)',image_level['id'].nunique(),
     image_level['StudyInstanceUID'].nunique())
display(image_level)
#glob.glob(os.path.join(train_images_loc,'5776db0cec75','*','*.dcm')) Work only on SUID
#glob.glob(os.path.join(train_images_loc,'*','*','000a312787f2.dcm')) Use id like this

In [ ]:
display(image_level[image_level.StudyInstanceUID=='0fd2db233deb'])
print(glob.glob(os.path.join(train_images_loc,'0fd2db233deb','*','*.dcm')))

In [ ]:
sns.set_style('darkgrid')
fig, axs = plt.subplots(figsize=(20,10),ncols=3)
image_level['boxes'].fillna(0,inplace=True)
image_level['count_boxes'] = image_level['boxes'].apply(lambda x:len(ast.literal_eval(x)) if x else 0)
sns.countplot(image_level.count_boxes,ax=axs[0])

image_level['label_type'] = image_level.label.str.split(expand=True)[0]
sns.countplot(image_level['label_type'],ax=axs[1])
#px.histogram(image_level.StudyInstanceUID.value_counts())


# fig = px.pie(image_level,values='label_type',names='count_boxes')
# fig.show()

In [ ]:
gg = image_level[image_level['StudyInstanceUID'].isin (image_level['StudyInstanceUID'].value_counts()[image_level['StudyInstanceUID'].value_counts()>1].index)]
display(gg[gg.label_type=='opacity'].groupby('StudyInstanceUID')['label_type'].describe())
display(gg[gg.label_type!='opacity'].groupby('StudyInstanceUID')['label_type'].describe())
print('Maximum Frequency of opacity instances for any StudyUID is ',gg[gg.label_type=='opacity'].groupby('StudyInstanceUID')['label_type'].describe()['freq'].max(),
     'and Maximum Frequency of None instances for any StudyUID is ',gg[gg.label_type!='opacity'].groupby('StudyInstanceUID')['label_type'].describe()['freq'].max())

MERGE THE DATAFRAME

Study level has 6054 rows and Image level has 6334 .

As in Image level we have multiple rows with same StudyInstanceUID.(0fd2db233deb)

In [ ]:
print(image_level.columns,study_level.columns)
merged_levels = pd.merge(image_level, study_level, left_on='StudyInstanceUID',right_on='id')
merged_levels['boxes'] = merged_levels['boxes'].apply(lambda x :ast.literal_eval(x) if x else x)
# merged_levels.drop(columns=['id_x','id_y'],inplace=True)
merged_levels.head()

So mutliple Rows with same SIUID seems to have maximum of only one row instance to be of type opacity and rest are None

In [ ]:
display(merged_levels[merged_levels['StudyInstanceUID']=='0fd2db233deb'])

Submission Files

Given a study id , find the image and make the prediction(class Confidence bbox)


In [ ]:
print(image_level.columns)
print(study_level.columns)

In [ ]:
sample_sub = pd.read_csv(sample_sub_loc)
print('Shape of Sample submission',sample_sub.shape)
sample_sub[['id','type']] = sample_sub['id'].str.split('_',expand=True)
display(sample_sub)
px.bar(sample_sub['type'].value_counts())

#glob.glob(os.path.join(test_images_loc,'00188a671292','*','*')) Works for study ids only
#glob.glob(os.path.join(test_images_loc,'*','*','46719b856de1.dcm')) For Image ids

Images 

In [ ]:
for i in merged_levels.StudyInstanceUID.unique():
    if len(glob.glob(os.path.join(train_images_loc,i,'*','*.dcm')))>8:
        print(i,len(glob.glob(os.path.join(train_images_loc,i,'*','*.dcm'))))

In [ ]:
def make_cord(boxes):
    boxes = ast.literal_eval(boxes)
    b = list()
    for b in boxes:
        x,y,w,h = j['x'],j['y'],j['width'],j['height']
        b.append(x,y,w,h)
    return b
def metadata(study_id):
    bbox,label =[],[]
    df = merged_levels[merged_levels.id_y==study_id][['boxes','type']]
    for i in df.iterrows():
        a,b = i[1][0],i[1][1]
        if a :
            bboxes = [[x['x'],x['y'],x['x']+x['width'],x['y']+x['height']] for x in a]
            bbox.append(bboxes)
        else:
            bbox.append([[-1,-1,-1,-1]])
        label.append(b)
        bbox
    return bbox,label

def show_img(studyid,figsize=20):
    if not isinstance(studyid, str):
        for i in studyid:
            show_img(i,figsize//len(studyid))
        return
    dcms = glob.glob(os.path.join(train_images_loc,studyid,'*','*.dcm'))
    row = ((len(dcms)-1)//3)+1
    ncols = 3 if len(dcms)>3 else len(dcms)
    fig,axes = plt.subplots(figsize=(figsize,figsize),ncols=ncols,nrows =row)
    meta_data = metadata(studyid)
    for cnt,path in enumerate(dcms):
        data = dicom.dcmread(path)
        msg = meta_data[1][cnt].split()[0]+' Body Part '+data['BodyPartExamined'].value +' SEX  '+ data["PatientSex"].value
        img = data.pixel_array
#         print(meta_data[0][cnt],meta_data[1][cnt])
#         Loc '/'.join(path.split('/')[-2:])
        if meta_data[0][cnt]:       
            for x1, y1, x2, y2 in meta_data[0][cnt]:
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                img = cv2.rectangle(img,(x1, y1), (x2, y2), color = [0,0,255], thickness = 10)
        if len(dcms) == 1:
            axes.imshow(img,cmap="gray")
            axes.axis('off')
            axes.title.set_text(msg)
        else:
            axes[cnt//3,cnt%3].imshow(img,cmap="gray")
            axes[cnt//3,cnt%3].axis('off')
            axes[cnt//3,cnt%3].title.set_text(msg)

In [ ]:
show_img('0fd2db233deb')

Some Images on ALL TYPES 

In [ ]:
for i in merged_levels[(merged_levels['type']=='Negative for Pneumonia')]['StudyInstanceUID'].unique()[:3]:
    show_img(i,20)

In [ ]:
for i in merged_levels[(merged_levels['count_boxes']>3)& (merged_levels['type']=='Typical Appearance')]['StudyInstanceUID'].unique()[:3]:
    show_img(i,4)

In [ ]:
for i in merged_levels[(merged_levels['count_boxes']>3)& (merged_levels['type']=='Typical Appearance')]['StudyInstanceUID'].unique()[:3]:
    show_img(i,4)

In [ ]:
show_img('53b92c44a02e',10)
show_img('341a8f794069',10)

In [ ]:
display(image_level[image_level['StudyInstanceUID'].isin(image_level.groupby('StudyInstanceUID').count()[image_level.groupby('StudyInstanceUID').count()['id']>1].index)])

DATA FROM IMAGE

os.path.join(train_images_loc,image_level.StudyInstanceUID[2],'*','*.dcm')

In [ ]:
# dicom.dcmread(glob.glob(os.path.join(train_images_loc,image_level.StudyInstanceUID[0],'*','*.dcm'))[0])
elem = dicom.dcmread(glob.glob(os.path.join(train_images_loc,image_level.StudyInstanceUID[2],'*','*.dcm'))[0])
for i in ['BodyPartExamined','ImageType',"PatientSex"]:
    print(elem[i])
elem

In [ ]:
display(elem)
print(elem.dir())

In [ ]:
attrs =['AccessionNumber', 'BitsAllocated', 'BitsStored', 'BodyPartExamined','Columns', 'DeidentificationMethod', 'HighBit', 'ImageType', 'ImagerPixelSpacing', 'InstanceNumber', 'Modality', 'PatientID', 'PatientName', 'PatientSex', 'PhotometricInterpretation', 'PixelRepresentation','Rows', 'SOPClassUID', 'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'SeriesNumber', 'SpecificCharacterSet', 'StudyDate', 'StudyID', 'StudyInstanceUID', 'StudyTime']
def get_metadata(path):
    elem = dicom.dcmread(path)
#     img = elem.get('PixelData')
    attrs =['AccessionNumber', 'BitsAllocated', 'BitsStored', 'BodyPartExamined','Columns', 'DeidentificationMethod', 'HighBit', 'ImageType', 'ImagerPixelSpacing', 'InstanceNumber', 'Modality', 'PatientID', 'PatientName', 'PatientSex', 'PhotometricInterpretation', 'PixelRepresentation','Rows', 'SOPClassUID', 'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'SeriesNumber', 'SpecificCharacterSet', 'StudyDate', 'StudyID', 'StudyInstanceUID', 'StudyTime']
    df = [path]
    for i in attrs:
        df.append(elem.get(i))
    return df

In [ ]:
# Datasets Saved
meta_data_loc = '../input/all-metadata/AllMeta.csv'
if os.path.isfile(meta_data_loc):
    full_meta = pd.read_csv(meta_data_loc,usecols=['path']+attrs)
else:
    full_meta = []
    for i in tqdm.tqdm(glob.glob(os.path.join(train_images_loc,'*','*','*.dcm'))):
        full_meta.append(get_metadata(i))
    full_meta = pd.DataFrame(full_meta,columns=['path']+attrs)
    full_meta.to_csv('AllMeta.csv')
    
    
full_meta_test = []
for i in tqdm.tqdm(glob.glob(os.path.join(test_images_loc,'*','*','*.dcm'))):
    full_meta_test.append(get_metadata(i))
full_meta_test = pd.DataFrame(full_meta_test,columns=['path']+attrs)
full_meta_test.to_csv('AllMeta_test.csv')

In [ ]:
test_images_loc

In [ ]:
print(attrs)

So around 56% percent entries aare of Men 

And most have examined their chest about 80%.

In [ ]:
#full_meta.groupby(['PatientSex','BodyPartExamined']).size().reset_index().pivot(columns='PatientSex',index='BodyPartExamined',values=0).plot(kind='bar', stacked=True)
print(full_meta.PatientSex.value_counts()/len(full_meta))
print(full_meta.BodyPartExamined.value_counts()/len(full_meta))
viz = full_meta.groupby(['PatientSex','BodyPartExamined']).size().reset_index().pivot(columns='PatientSex',index='BodyPartExamined',values=0)
px.bar(viz,y=['F','M'])

In [ ]:
for i in full_meta.columns:
    print(i,full_meta[i].nunique())

In [ ]:
merged_meta = pd.merge(merged_levels,full_meta,left_on='id_x',right_on='SOPInstanceUID')
display(merged_meta)

In [ ]:
viz = merged_meta.groupby(['type','BodyPartExamined']).size().reset_index().pivot(columns='BodyPartExamined',index='type',values=0)
px.bar(viz,orientation='h')

Multiple Patients have same Name , so Even though their identity are anonymous the names have been encoded in a particular way.

About 60% percent of patients have only one entry and as 96% of the total patients have exactly 1 image of them , we can say for sure that a good size of sample around (40-4 = 36%) of entries are made by a person who had a test before .

Let me explain this in simple step:
1. 96-4 is the ratio of patient having multiple images of them ,(as multiple images are a product of artifacts{small problems while taking the image } .
2. 60-40 is the ratio of patiens having single entries ,but already 4% of entries have more than 1 image so that information is of not much use.
3. So the main calculation here is that the 4% out of that 40% are the ones have multiple images due to error so we have 36% of actual double entries .

In [ ]:
px.histogram(merged_meta['PatientName'].value_counts(),histnorm='percent')

In [ ]:
from pandas_profiling import ProfileReport
profile_study = ProfileReport(merged_levels, title="Pandas Profiling Report - Train Study df")
profile_study.to_widgets()